In [48]:
%pip install pandas scikit-learn openpyxl xlrd
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Excelデータの読み込み
file_path = '/Users/yasudayuuya/BLP_data/【Aラインのみ】生産性結果記入用紙_Ver.4.3.xlsx'  # Excelファイルのパスを指定
sheet_name = '実績昼'  # 読み込むシート名を指定

data = pd.read_excel(file_path, sheet_name=sheet_name)
data.head()


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,作業ライン,項目,2024-11-01 00:00:00,2024-11-02 00:00:00,2024-11-03 00:00:00,2024-11-04 00:00:00,2024-11-05 00:00:00,2024-11-06 00:00:00,2024-11-07 00:00:00,2024-11-08 00:00:00,...,2024-11-24 00:00:00,2024-11-25 00:00:00,2024-11-26 00:00:00,2024-11-27 00:00:00,2024-11-28 00:00:00,2024-11-29 00:00:00,2024-11-30 00:00:00,2024-12-01 00:00:00,合計,NaN
2,A-PaLS,件数,0,2199,0,0,1172,1894,2912,2794,...,1332,2867,3622,3856,2360,1597,1582,0,59330,NaN
3,NaN,作業時間(分),0,5262,0,0,2070,3567,8088,7374,...,4302,9009,7893,9682,8725,7200,4905,0,172247,NaN
4,NaN,目標件数,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
# 行列を逆にする
data = data.T
# 1行目を列名にする
data.columns = data.iloc[1]
# 0,1行目を削除
data = data.drop(data.index[[0, 1]])
data.head()


Unnamed: 1,NaN,項目,件数,作業時間(分),目標件数,作業人数,リーダー人数,予測,買い合わせ,NaN,項目,生産性(分/件),1日作業量/L
Unnamed: 2,NaN,2024-11-01 00:00:00,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0
Unnamed: 3,NaN,2024-11-02 00:00:00,2199,5262,NaN,13,5,NaN,NaN,NaN,2,2.392906,181.787343
Unnamed: 4,NaN,2024-11-03 00:00:00,0,0,NaN,NaN,NaN,NaN,NaN,NaN,3,0,0
Unnamed: 5,NaN,2024-11-04 00:00:00,0,0,NaN,NaN,NaN,NaN,NaN,NaN,4,0,0
Unnamed: 6,NaN,2024-11-05 00:00:00,1172,2070,NaN,NaN,6,NaN,NaN,NaN,5,1.766212,246.289855


In [50]:
# 欠損値を探す
data.isnull().sum()


Unnamed: 1
NaN         33
項目           1
件数           1
作業時間(分)      1
目標件数        33
作業人数         8
リーダー人数       7
予測          33
買い合わせ       33
NaN         33
項目           0
生産性(分/件)     0
1日作業量/L      0
dtype: int64

In [51]:
# 0,1,4,7,8,9列目を削除
data = data.drop(data.columns[[0, 1, 4, 7, 8, 9]], axis=1)
data.head()

Unnamed: 1,件数,作業時間(分),作業人数,リーダー人数,生産性(分/件),1日作業量/L
Unnamed: 2,0,0,NaN,NaN,0,0
Unnamed: 3,2199,5262,13,5,2.392906,181.787343
Unnamed: 4,0,0,NaN,NaN,0,0
Unnamed: 5,0,0,NaN,NaN,0,0
Unnamed: 6,1172,2070,NaN,6,1.766212,246.289855


In [52]:
# NaNを全て0に変換
data = data.fillna(0)
data.head()

/var/folders/wl/_sk4lq9107v79smv_pqvls780000gn/T/ipykernel_69908/287054918.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.fillna(0)


Unnamed: 1,件数,作業時間(分),作業人数,リーダー人数,生産性(分/件),1日作業量/L
Unnamed: 2,0,0,0,0,0.000000,0.000000
Unnamed: 3,2199,5262,13,5,2.392906,181.787343
Unnamed: 4,0,0,0,0,0.000000,0.000000
Unnamed: 5,0,0,0,0,0.000000,0.000000
Unnamed: 6,1172,2070,0,6,1.766212,246.289855


In [53]:
# 0列目のデータを表示
data.iloc[:, 0]
data

Unnamed: 1,件数,作業時間(分),作業人数,リーダー人数,生産性(分/件),1日作業量/L
Unnamed: 2,0,0,0,0,0.000000,0.000000
Unnamed: 3,2199,5262,13,5,2.392906,181.787343
Unnamed: 4,0,0,0,0,0.000000,0.000000
Unnamed: 5,0,0,0,0,0.000000,0.000000
Unnamed: 6,1172,2070,0,6,1.766212,246.289855
Unnamed: 7,1894,3567,5,4,1.883316,230.975610
Unnamed: 8,2912,8088,17,4,2.777473,156.617211
Unnamed: 9,2794,7374,17,4,2.639227,164.820993
Unnamed: 10,3176,8283,13,2,2.607997,166.794640
Unnamed: 11,0,0,0,0,0.000000,0.000000


In [54]:
# 33,34行目を削除
data = data.drop(data.index[[31, 32]])
data

Unnamed: 1,件数,作業時間(分),作業人数,リーダー人数,生産性(分/件),1日作業量/L
Unnamed: 2,0,0,0,0,0.000000,0.000000
Unnamed: 3,2199,5262,13,5,2.392906,181.787343
Unnamed: 4,0,0,0,0,0.000000,0.000000
Unnamed: 5,0,0,0,0,0.000000,0.000000
Unnamed: 6,1172,2070,0,6,1.766212,246.289855
Unnamed: 7,1894,3567,5,4,1.883316,230.975610
Unnamed: 8,2912,8088,17,4,2.777473,156.617211
Unnamed: 9,2794,7374,17,4,2.639227,164.820993
Unnamed: 10,3176,8283,13,2,2.607997,166.794640
Unnamed: 11,0,0,0,0,0.000000,0.000000


In [62]:
# 正規化 -1から1の範囲に収める  
data2 = data.astype(float) # データ型をfloatに変換
data2 = (data - data.mean()) / data.std() # 正規化。各列の値から平均を引いて標準偏差で割る
data2

Unnamed: 1,件数,作業時間(分),作業人数,リーダー人数,生産性(分/件),1日作業量/L
Unnamed: 2,-1.695705,-1.748705,-1.588297,-1.265237,-1.795602,-1.782370
Unnamed: 3,0.252627,-0.092640,0.114053,0.756533,-0.000189,0.827053
Unnamed: 4,-1.695705,-1.748705,-1.588297,-1.265237,-1.795602,-1.782370
Unnamed: 5,-1.695705,-1.748705,-1.588297,-1.265237,-1.795602,-1.782370
Unnamed: 6,-0.657304,-1.097231,-1.588297,1.160887,-0.470402,1.752939
Unnamed: 7,-0.017606,-0.626093,-0.933547,0.352179,-0.382538,1.533114
Unnamed: 8,0.884350,0.796763,0.637853,0.352179,0.288353,0.465754
Unnamed: 9,0.779801,0.572052,0.637853,0.352179,0.184627,0.583513
Unnamed: 10,1.118256,0.858134,0.114053,-0.456529,0.161195,0.611844
Unnamed: 11,-1.695705,-1.748705,-1.588297,-1.265237,-1.795602,-1.782370


In [63]:
# 目的変数と説明変数の設定
# 'target_column'を目的変数として、他のすべてのカラムを説明変数とする
X = data2.drop(columns= '件数')
y = data2['件数']

# データセットをトレーニングセットとテストセットに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ランダムフォレスト回帰モデルの構築
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 予測
y_pred = model.predict(X_test)

# モデルの評価
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R² Score: {r2}')

Mean Squared Error: 0.057992263646625845
R² Score: 0.9203456970146162


In [64]:
# 実際に件数を予測
X_new = X_test.iloc[0].values.reshape(1, -1)
y_new = model.predict(X_new)
# Predictedは予測値、Actualは実際の値、Errorは予測値と実際の値の差
print(f'Predicted: {y_new[0]}')
print(f'Actual: {y_test.iloc[0]}')
print(f'Error: {y_test.iloc[0] - y_new[0]}')
# 正規化する前の値に戻す
X_new = X_new * data.drop(columns='件数').std().values + data.drop(columns='件数').mean().values
y_new = y_new * data['件数'].std() + data['件数'].mean()
y_test = y_test * data['件数'].std() + data['件数'].mean()
print(f'Predicted: {y_new[0]}')
print(f'Actual: {y_test.iloc[0]}')
print(f'Error: {y_test.iloc[0] - y_new[0]}')

Predicted: 0.3399869462649558
Actual: 0.39527383796061905
Error: 0.055286891695663254
Predicted: 2297.6
Actual: 2360.0
Error: 62.40000000000009


/Users/yasudayuuya/.pyenv/versions/3.10.1/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [70]:
# 特徴量重要度を表示
feature_importances = model.feature_importances_
features = X.columns
df = pd.DataFrame({'features': features, 'importance': feature_importances})
df = df.sort_values(by='importance', ascending=False)
df

,features,importance
0,作業時間(分),0.319397
3,生産性(分/件),0.287233
1,作業人数,0.176469
4,1日作業量/L,0.150034
2,リーダー人数,0.066867
